In [ ]:
import dask.array as da
import warnings

import dask.array as da
from sklearn.pipeline import Pipeline
from dask_ml.model_selection import train_test_split
from dask_ml.model_selection import GridSearchCV
from dask_ml.decomposition import PCA
from dask_ml.preprocessing import PolynomialFeatures
from dask_ml.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import pandas
from toolz.curried import groupby, valmap, pipe, pluck, merge_with, merge
from toolz.curried import map as fmap

from pymks.fmks.data.elastic_fe import solve
from pymks.fmks.data.multiphase import generate

from pymks.fmks.plot import plot_microstructures
from pymks.fmks.bases.primitive import PrimitiveTransformer
from pymks.fmks.correlations import TwoPointCorrelation, FlattenTransformer
from dask.distributed import Client, progress
import time
from mpl_toolkits import mplot3d
%matplotlib notebook
%load_ext memory_profiler

In [ ]:
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
da.random.seed(10)
np.random.seed(10)

tmp = [
    generate(shape=(100, 101, 101,101), grain_size=x, volume_fraction=(0.5, 0.2,0.3), chunks=25, percent_variance=0.15)
    for x in [(15, 2,2), (2,2, 15), (7,7, 7), (9, 9,3), (9,9, 9), (2, 2,2)]
]
x_data_gen = da.concatenate(tmp)

In [ ]:
@profile
def HomogenizationPipeline(x):
    a1=PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0).transform(x)
    a2=TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)]).transform(a1)
    a3=FlattenTransformer().transform(a2)
    a4=PCA(n_components=3).fit_transform(a3)
    return a4




In [3]:
%%file memory_try.py

import dask.array as da
import warnings

import dask.array as da
from sklearn.pipeline import Pipeline
from dask_ml.model_selection import train_test_split
from dask_ml.model_selection import GridSearchCV
from dask_ml.decomposition import PCA
from dask_ml.preprocessing import PolynomialFeatures
from dask_ml.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import pandas
from toolz.curried import groupby, valmap, pipe, pluck, merge_with, merge
from toolz.curried import map as fmap

from pymks.fmks.data.elastic_fe import solve
from pymks.fmks.data.multiphase import generate

from pymks.fmks.plot import plot_microstructures
from pymks.fmks.bases.primitive import PrimitiveTransformer
from pymks.fmks.correlations import TwoPointCorrelation, FlattenTransformer
from dask.distributed import Client, progress
import time
from mpl_toolkits import mplot3d


da.random.seed(10)
np.random.seed(10)

tmp = [
    generate(shape=(100, 101, 101,101), grain_size=x, volume_fraction=(0.5, 0.2,0.3), chunks=25, percent_variance=0.15)
    for x in [(15, 2,2), (2,2, 15), (7,7, 7), (9, 9,3), (9,9, 9), (2, 2,2)]
]
x_data_gen = da.concatenate(tmp)


@profile
def HomogenizationPipeline(x):
    a1=PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0).transform(x)
    a2=TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)]).transform(a1)
    a3=FlattenTransformer().transform(a2)
    a4=PCA(n_components=3).fit_transform(a3)
    return a4

HomogenizationPipeline(x_data_gen)


Overwriting memory_try.py


In [5]:
%%file memory_try2.py

import dask.array as da
import warnings

import dask.array as da
from sklearn.pipeline import Pipeline
from dask_ml.model_selection import train_test_split
from dask_ml.model_selection import GridSearchCV
from dask_ml.decomposition import PCA
from dask_ml.preprocessing import PolynomialFeatures
from dask_ml.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
import pandas
from toolz.curried import groupby, valmap, pipe, pluck, merge_with, merge
from toolz.curried import map as fmap

from pymks.fmks.data.elastic_fe import solve
from pymks.fmks.data.multiphase import generate

from pymks.fmks.plot import plot_microstructures
from pymks.fmks.bases.primitive import PrimitiveTransformer
from pymks.fmks.correlations import TwoPointCorrelation, FlattenTransformer
from dask.distributed import Client, progress
import time
from mpl_toolkits import mplot3d


da.random.seed(10)
np.random.seed(10)

tmp = [
    generate(shape=(100, 101, 101,101), grain_size=x, volume_fraction=(0.5, 0.2,0.3), chunks=25, percent_variance=0.15)
    for x in [(15, 2,2), (2,2, 15), (7,7, 7), (9, 9,3), (9,9, 9), (2, 2,2)]
]
x_data_gen = da.concatenate(tmp)


@profile
def HomogenizationPipeline(x):
    a1=PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0).transform(x).compute()
    a2=TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)]).transform(a1).compute()
    a3=FlattenTransformer().transform(a2).compute()
    a4=PCA(n_components=3).fit_transform(a3).compute()
    return a4

HomogenizationPipeline(x_data_gen)

Writing memory_try2.py


In [4]:
!python -m memory_profiler memory_try.py

Filename: memory_try.py

Line #    Mem usage    Increment   Line Contents
    39  183.355 MiB  183.355 MiB   @profile
    40                             def HomogenizationPipeline(x):
    41  183.441 MiB    0.086 MiB       a1=PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0).transform(x)
    42  184.012 MiB    0.570 MiB       a2=TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)]).transform(a1)
    43  184.012 MiB    0.000 MiB       a3=FlattenTransformer().transform(a2)
    44 9977.887 MiB 9793.875 MiB       a4=PCA(n_components=3).fit_transform(a3)
    45 9977.887 MiB    0.000 MiB       return a4




In [ ]:
!python -m memory_profiler memory_try2.py

In [ ]:
a1=PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0).transform(x_data_gen)


In [ ]:
a2=TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)]).transform(a1)
a2

In [ ]:
a3=FlattenTransformer().transform(a2)
a3

In [ ]:
a4=PCA(n_components=3).transform(a3)

In [ ]:
# client = Client( threads_per_worker=2,
#                 n_workers=2, memory_limit='8GB')
# client

In [ ]:
bl1 = Pipeline(steps=[
    ("discritize",PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0)),
    ("Correlations",TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)])),
    ('flatten', FlattenTransformer()),  
    ('reducer',PCA(n_components=3))])

In [ ]:
tic = time.time()
%memit a=bl1.fit(x_data_gen)
toc = time.time()
print(tic-toc)

In [ ]:
tic = time.time()
%memit b=bl1.transform(x_data_gen).compute()
toc = time.time()
print(tic-toc)
print(b)

In [ ]:
da.random.seed(10)
np.random.seed(10)
tmp2 = [
    generate(shape=(100, 101, 101,101), grain_size=x, volume_fraction=(0.5, 0.2,0.3), chunks=20, percent_variance=0.15)
    for x in [(15, 2,2), (2,2, 15), (7,7, 7), (9, 9,3), (9,9, 9), (2, 2,2)]
]
x_data_gen2 = da.concatenate(tmp2)

In [ ]:
tic = time.time()
%memit a2=bl1.fit(x_data_gen2)
toc = time.time()
print(tic-toc)

In [ ]:
tic = time.time()
%memit b2=bl1.transform(x_data_gen2).compute()
toc = time.time()
print(tic-toc)
print(b2)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection="3d")

# z_line = np.linspace(0, 15, 1000)
# x_line = np.cos(z_line)
# y_line = np.sin(z_line)
# ax.plot3D(x_line, y_line, z_line, 'gray')

z_points = b[:,2]
x_points = b[:,0]
y_points = b[:,1]
ax.scatter3D(x_points, y_points, z_points, c=z_points, cmap='hsv');

plt.show()

In [ ]:

fig = plt.figure()
ax = plt.axes(projection="3d")

# z_line = np.linspace(0, 15, 1000)
# x_line = np.cos(z_line)
# y_line = np.sin(z_line)
# ax.plot3D(x_line, y_line, z_line, 'gray')

z_points = b2[:,2]
x_points = b2[:,0]
y_points = b2[:,1]
ax.scatter3D(x_points, y_points, z_points, c=z_points, cmap='hsv');

plt.show()
